ابتدا کلاس و متد های مورد نیاز را پیاده سازی میکنیم:

کلاس مورد نیاز برای نگه داشتن یک مجموعه و مقدار ساپورت آن در قالب یک موجودیت:

In [ ]:
class SupportfulSet:
    def __init__(self, the_set, support_count):
        self.the_set = the_set
        self.support = support_count

    def __str__(self):
        return f'SET: {self.the_set} | Sup. Count: {self.support}'

تابع برای تولید اولین مجموعه ورودی الگوریتم برای تولید لیستی از آبجکت های شامل هر آیتم و ساپورت آنها به صورت مستقل:

In [ ]:
def get_l1(input: set) -> list:
    dict_result = dict()

    for transaction in input:
        for item in transaction:
            if item in dict_result:
                dict_result[item] += 1
            else:
                dict_result[item] = 1

    result = list()
    for item in dict_result:
        result.append(SupportfulSet([item], dict_result[item]))

    return result

تابعی برای جوین زدن اعضای یک مجموعه با یکدیگر به شرطی که در یک عضو با هم اختلاف داشته باشند:

In [ ]:
def perform_join(input: set) -> list:

    if(type(input) is dict):
        input = [key for key in input]

    result = []

    for item in input:
        for other_item in input:
            if other_item != item:
                union_set = set()

                if type(other_item) is str:
                    union_set = { other_item, item }
                elif len(set(other_item) - set(item)) == 1:
                    union_set = set(item) | set(other_item)

                if len(union_set) > 0 and union_set not in result:
                    result.append(union_set)

    return result

تابع مورد نیاز برای حذف آبجکت هایی که مقدار ساپورت مجموعه ی آنها، از حد نساب کمتر است:

In [ ]:
def remove_items_with_invalid_support(input: list, min_sup: int) -> list:
    return { SupportfulSet(object.the_set, object.support) for object in input if object.support >= min_sup }

تابعی برای تولید زیر مجموعه های یک مجموعه:

In [ ]:
def get_subsets(input: list, subset_length: int):
    import math

    input = list(input)
    # معادل باینری زیر مجموعه ها را بدست می اوریم
    binaries = [bin(item)[2:] for item in range(0, int(math.pow(2, len(input)))) if bin(item)[2:].count('1') == subset_length]
    # خروجی دستور بالا، دارای طول به اندازه تعداد اعضای مجموعه ورودی نیست. پس آن را نرمال میکنیم
    normalized_binaries = []

    for binary in binaries:
        temp = binary
        while len(temp) < len(input):
            temp = '0' + temp
        normalized_binaries.append(temp)

    result = []

    # بر روی رشته های باینری معادل حلقه زده و به ازای هر رشته باینری
    # هر جا عدد یک مشاهده شد، عضو نظیر آن در مجموعه ورودی را در یک زیر مجموعه اضافه میکنیم
    for binary in normalized_binaries:
        subset = []
        for index in range(len(binary)):
            if binary[index] == '1':
                subset.append(input[index])
        result.append(subset)

    return result


تابعی برای حذف مجموعه هایی که زیر مجموعه های دارای یک عضو کمتر از خود مجموعه، در مجموعه کاندید قبلی حضور ندارند:

In [ ]:
def remove_items_with_nonfrequent_subsets(candidate_list: list, previous_level_frequents: dict) -> dict:

    must_removed_items = []

    previous_level_subsets = [list(obj.the_set) for obj in previous_level_frequents]

    for superset in candidate_list:
        subsets = get_subsets(superset, len(superset) - 1)
        for subset in subsets:
            if type(subset) is str:
                subset = [subset]
            if subset not in previous_level_subsets:
                must_removed_items.append(superset)

    return [candidate for candidate in candidate_list if candidate not in must_removed_items]

الحاق مقدار ساپورت مجموعه ورودی و برگرداندن شیء از جنس کلاس تعریف شده مخصوص این کار با استفاده از دیتاست مورد استفاده الگوریتم:

In [ ]:
def append_support(candidate_list: list, dataset: list) -> dict:
    result = list()

    for candidate in candidate_list:
        candidate_obj = SupportfulSet(candidate, 0)
        for transaction in dataset:
            if set(candidate) <= set(transaction):
                candidate_obj.support += 1
        result.append(candidate_obj)

    return result

و در نهایت استفاده از تابع های نوشته شده و نوشتن الگوریتم آپریوری:

In [ ]:
def perform_apriori(input: list, min_sup: int) -> list:

    if(type(input) is not list):
        raise TypeError('list parameter passed to Apriori must be of type list')

    if(type(min_sup) is not int):
        raise TypeError('min_sup parameter passed to Apriori must be of type int')


    found_frequent_set = False
    l_k = get_l1(input) # l_k is a list of supportful (has support property :D)  sets

    while not found_frequent_set:
        c_new = perform_join([object.the_set for object in l_k]) # C_new = k+1-th Candidate set & is a set (not supportful)

        c_new = remove_items_with_nonfrequent_subsets(c_new, l_k)

        l_k = append_support(c_new, input)

        pruned_l_k = remove_items_with_invalid_support(l_k, min_sup)

        if(len(pruned_l_k) == 0):
            found_frequent_set = True
        else:
            l_k = pruned_l_k

    return l_k


پیش از اجرای الگوریتم، تابع دیگری نیز نیاز داریم تا فایل دیتاست را به صورت لیستی از تراکنش ها بخوانیم.

**نکته:**


این روش نیازمند فایل با پسوند


xlsx


میباشد که در کگل یافت نشد
به ناچار به صورت لوکال آپلود میشود.
اما بنده دیتای مورد نظر را در لینک زیر نیز آپلود کرده ام که برای دیگران قابل دانلود میباشد:


https://s32.picofile.com/d/8480032150/ec89d384-9cd3-4cb4-a426-86b4936b6d79/games_sales_dataset.xlsx

In [ ]:
def get_data():

    import openpyxl
    
    file_path = input('Enter the path to the Excel file (.xlsx): ')

    workbook = openpyxl.load_workbook(file_path)

    sheet = workbook.active
    transactions = []

    for row in sheet.iter_rows(values_only=True):
        transaction = []
        for cell in row:
            if cell is not None:
                transaction.append(str(cell))
        transactions.append(transaction)

    return transactions

در اینجا دیتاست مورد نیاز را واکشی و به الگوریتم ارسال میکنیم:

دیتاست انتخاب شده در این تمرین، تراکنش های مربوط به خرید بازی های رایانه ای است

In [ ]:
transactions = get_data()
print(f'تعداد تراکنش ها: {len(transactions)}')
print('چند رکورد از تراکنش های واکشی شده:')
print(transactions[:10])

Saving games_sales_dataset.xlsx to games_sales_dataset (3).xlsx
تعداد تراکنش ها: 12526
چند رکورد از تراکنش های واکشی شده:
[['God of War', 'The Last of Us', 'Read Dead Redemption', 'Minecraft', 'Grand Theft Auto V', 'Left 4 Dead'], ['Grand Theft Auto V', 'The Last of Us'], ['God of War', "Assassin's Creed 2", 'Read Dead Redemption', 'Left 4 Dead'], ['Left 4 Dead', "Assassin's Creed 2", 'Super Mario World', 'The Last of Us', 'Read Dead Redemption', 'The Elder Scrolls V: Skyrim'], ['Left 4 Dead', 'Minecraft', 'The Last of Us', 'Dark Souls', 'Read Dead Redemption', 'Resident Evil 4'], ['Dark Souls', "Assassin's Creed 2", 'Grand Theft Auto V', 'Resident Evil 4', 'Super Mario World', 'Guitar Hero 3'], ['Resident Evil 4', 'The Last of Us'], ['Read Dead Redemption', 'Grand Theft Auto V', 'Super Mario World'], ['Resident Evil 4', "Assassin's Creed 2", 'Read Dead Redemption', 'Minecraft', 'Dark Souls', 'Left 4 Dead'], ['Super Mario World', 'Grand Theft Auto V']]


اجرای الگوریتم و چاپ مجموعه های دارای الگوی تکراری با حداقل ساپورت 2500
علت مقدار 2500 این است که بعد از چندین مرحله اجرای الگوریتم، مقدار ساپورت حدودا بین 2400 تا 2600 میشود
و بیشتر از آن موجب حذف تمامی مجموعه ها و کمتر از آن موجب تعدد  مجموعه ها میگردد:

In [ ]:
result = perform_apriori(transactions, 2500)

print('مجموعه های پرتکرار:')
for obj in result:
  print(obj.the_set)

print(f'تعداد : {len(result)}')

مجموعه های پرتکرار:
{'God of War', 'The Last of Us', 'Dark Souls'}
{'God of War', 'Dark Souls', 'Grand Theft Auto V'}
{'God of War', 'Dark Souls', 'Read Dead Redemption'}
{'God of War', 'Dark Souls', 'Guitar Hero 3'}
{'God of War', 'The Last of Us', 'Grand Theft Auto V'}
{'God of War', 'The Last of Us', 'Guitar Hero 3'}
{'God of War', 'The Last of Us', 'Read Dead Redemption'}
{'Dark Souls', 'Grand Theft Auto V', 'Super Mario World'}
{'The Elder Scrolls V: Skyrim', 'The Last of Us', 'Grand Theft Auto V'}
{'The Last of Us', 'Guitar Hero 3', 'Grand Theft Auto V'}
{'The Last of Us', 'Dark Souls', 'Grand Theft Auto V'}
{'The Last of Us', 'Grand Theft Auto V', 'Read Dead Redemption'}
{'The Elder Scrolls V: Skyrim', 'Left 4 Dead', 'The Last of Us'}
{'The Elder Scrolls V: Skyrim', 'The Last of Us', 'Guitar Hero 3'}
{'The Elder Scrolls V: Skyrim', 'The Last of Us', 'Minecraft'}
{'God of War', 'The Elder Scrolls V: Skyrim', 'Guitar Hero 3'}
{'God of War', 'The Elder Scrolls V: Skyrim', 'Grand Th